# Assignment 1C - Question 2
## Semantic Person Search

In [1]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
import keras
from keras import layers


import os
import datetime
import numpy

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorboard import notebook
from tensorflow.keras.preprocessing.image import Iterator

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

import pydot
import IPython
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot, plot_model

import cv2

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
train = pd.read_csv('CAB420_Assessment_1C_Data/Data/Q2/Q2/Train_Data/Train.csv')
test = pd.read_csv('CAB420_Assessment_1C_Data/Data/Q2/Q2/Test_Data/Test.csv')

In [3]:
train_img = []
gnd_img = []
files = glob.glob('CAB420_Assessment_1C_Data/Data/Q2/Q2/Train_Data/Originals/*.png')
for myfile in files:
    image = cv2.imread(myfile, 0)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gnd_img.append(myfile[58:])
    train_img.append(image)

#train_img = np.array(train_img)
gnd_img = np.array(gnd_img)

#print('Training shape: ', train_img.shape)
#print('Training gnd: ', gnd_img[1:10])

In [4]:
test_originals = []
gnd_test = []
files = glob.glob('CAB420_Assessment_1C_Data/Data/Q2/Q2/Test_Data/Originals/*.png')
for myfile in files:
    image = cv2.imread(myfile)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gnd_test.append(myfile[57:])
    test_originals.append(image)

#test_originals = np.array(test_originals)
gnd_test = np.array(gnd_test)
#print('Testing shape: ', test_originals.shape)
#print('Testing gnd: ', gnd_test[1:10])

In [5]:
train = train.drop(columns=['torcol2','torcol3','tortex','torcol3','legcol2','legcol3','legtex','pose'])
train_gender = train.iloc[:1]
train_tortyp = train.iloc[:2]
train_torcol = train.iloc[:3]
train_legtyp = train.iloc[:4]
train_legcol = train.iloc[:5]
train_luggage = train.iloc[:6] 



In [6]:
test = test.drop(columns=['torcol2','torcol3','tortex','torcol3','legcol2','legcol3','legtex','pose'])
test_gender = test.iloc[:1]
test_tortyp = test.iloc[:2]
test_torcol = test.iloc[:3]
test_legtyp = test.iloc[:4]
test_legcol = test.iloc[:5]
test_luggage = test.iloc[:6] 

In [7]:
#fig = plt.figure(figsize=[20, 20])
#for i in range(100):
#    ax = fig.add_subplot(10, 10, i + 1)
#    ax.imshow(test_originals[i])

In [8]:
inputs = keras.Input(shape=(144, 226, 3, ), name='img')
x = layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu')(inputs)
x = layers.MaxPool2D(pool_size=(2, 2))(x)
x = layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu')(x)
x = layers.MaxPool2D(pool_size=(2, 2))(x)
x = layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
x = layers.Flatten()(x)
x1 = layers.Dense(64, activation='relu')(x)
gender = layers.Dense(3, name='gender_out')(x1)

x2 = layers.Dense(64, activation='relu')(x)
tortyp = layers.Dense(3, name='tortyp_out')(x2)

x3 = layers.Dense(64, activation='relu')(x)
torcol = layers.Dense(11, name='torcol_out')(x3)

x4 = layers.Dense(64, activation='relu')(x)
legtyp = layers.Dense(3, name='legtyp_out')(x4)

x5 = layers.Dense(64, activation='relu')(x)
legcol = layers.Dense(11, name='legocol_out')(x5)

x6 = layers.Dense(64, activation='relu')(x)
luggage = layers.Dense(3, name='luggage_out')(x6)

model_cnn = keras.Model(inputs=inputs, outputs=[gender,tortyp,torcol,legtyp,legcol,luggage], name='A1CQ2')
model_cnn.summary()

Model: "A1CQ2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 144, 226, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 142, 224, 8)  224         img[0][0]                        
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 71, 112, 8)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 69, 110, 16)  1168        max_pooling2d[0][0]              
______________________________________________________________________________________________

In [9]:
model_cnn.compile(loss=['mean_squared_error', keras.losses.SparseCategoricalCrossentropy(from_logits=True)],
              optimizer=keras.optimizers.RMSprop())

In [10]:
history = model_cnn.fit(train_img, [train_gender, train_tortyp,train_torcol,train_legtyp,train_legcol,train_luggage],
                    batch_size=64,
                    epochs=20,
                    validation_data=(test_originals, [test_gender, test_tortyp, test_torcol, test_legtyp, test_legcol, test_luggage]))

ValueError: Data cardinality is ambiguous:
  x sizes: 226, 207, 220, 188, 249, 193, 275, 145, 206, 393, 231, 155, 189, 190, 216, 202, 171, 233, 189, 172, 157, 187, 257, 204, 225, 167, 180, 191, 240, 214, 209, 199, 191, 205, 145, 219, 179, 218, 188, 205, 210, 179, 188, 211, 197, 181, 236, 186, 166, 218, 150, 187, 182, 166, 229, 204, 157, 250, 216, 239, 212, 179, 187, 295, 159, 188, 187, 213, 188, 249, 199, 273, 185, 187, 153, 188, 178, 177, 244, 234, 243, 207, 198, 199, 249, 228, 200, 190, 260, 257, 201, 217, 219, 248, 244, 231, 155, 191, 228, 208, 247, 212, 230, 196, 207, 224, 257, 248, 176, 219, 222, 223, 221, 214, 211, 168, 214, 270, 243, 236, 200, 180, 192, 189, 178, 228, 189, 222, 192, 138, 170, 209, 199, 182, 214, 338, 307, 210, 204, 269, 211, 210, 186, 214, 213, 215, 188, 163, 194, 280, 200, 220, 210, 177, 300, 256, 176, 194, 209, 184, 214, 199, 202, 194, 279, 210, 249, 181, 188, 294, 218, 190, 221, 293, 176, 168, 207, 201, 204, 206, 233, 231, 273, 345, 353, 253, 142, 189, 177, 1